
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

In [11]:
import numpy as np
import pandas as pd
import json
import os
from collections import namedtuple
from collections import defaultdict as dd
from math import log
from sklearn.metrics import confusion_matrix
import random

#TODO: REMOVE THIS 
import os
os.chdir("/home/rohan/Repositories/comp30027-machine-learning/ass1/src/submission")

In [ ]:
RANDOM_SEED = 2
NUM_PARTITIONS = 10


In [18]:
def preprocess(filepath, header=None, config_file="config.json"):

    #load configuration settings from file
    with open(config_file) as fp:
        config = json.load(fp)

    #extract the name of the dataset from the filepath
    pth, fname = os.path.split(filepath)    #pylint:disable=unused-variable
    fname, ext = os.path.splitext(fname)    #pylint:disable=unused-variable

    if fname not in config:
        raise NotImplementedError(f"The \'{fname}\' dataset is not supported.")
    
    data_config = config[fname]

    print(data_config)

    df = pd.read_csv(
        filepath, 
        header=header, 
        names=range(data_config['columns']),
        na_values=data_config['missing_values']
    )
    
    df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    
    return df, data_config

{'instances': 8124, 'columns': 23, 'class_col': 0, 'missing_values': ['?'], 'discrete': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22], 'numeric': []}
Data summary: -------------------------------------------------------
0: (class) dtype=object
values (2): ['p' 'e']
missing: 0 / 8124

1: (discrete) dtype=object
values (6): ['x' 'b' 's' 'f' 'k' 'c']
missing: 0 / 8124

2: (discrete) dtype=object
values (4): ['s' 'y' 'f' 'g']
missing: 0 / 8124

3: (discrete) dtype=object
values (10): ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
missing: 0 / 8124

4: (discrete) dtype=object
values (2): ['t' 'f']
missing: 0 / 8124

5: (discrete) dtype=object
values (9): ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
missing: 0 / 8124

6: (discrete) dtype=object
values (2): ['f' 'a']
missing: 0 / 8124

7: (discrete) dtype=object
values (2): ['c' 'w']
missing: 0 / 8124

8: (discrete) dtype=object
values (2): ['n' 'b']
missing: 0 / 8124

9: (discrete) dtype=object
values (12): ['k' 'n' 'g'

In [ ]:
""" Convenient way to store the data for a learned Naive Bayes model. 

    Fields:
        discrete:     discrete probability data. A triple-nested dictionary
                      such that discrete[da][c][x] = P(x|c) where x is a 
                      value of a discrete attribute da.
        numeric:      numeric probability data. A dictionary (keyed by attribute 
                      name) of tuples (mu, sigma) where each is an array of the 
                      same length as class_vals storing the mean and standard 
                      deviation of this attribute for each class value.
        class_vals:   an array of possible values of the class to predict
        class_priors: an array the same length of class_vals such that the ith
                      element is the probability of class_vals[i]
"""
NBModel = namedtuple("NBModel", ['discrete', 'numeric',
                                 'class_vals', 'class_priors'])

def discrete_probabilities(obs, vals):
    """ Estimates the probability of observing each value of a discrete 
        phenomena, based on a series of observations.

        Args:
            obs: a numpy array of observations where each element is contained 
            in vals
            vals: an iterable of possibly observable values

        Returns:
            A numpy array a where a[i] is the probability of observing vals[i]
    """
    return np.array([np.count_nonzero((obs == v)) / len(obs) for v in vals])


def laplace_smoothing(n_attr_obs, n_class_obs, n_attr_vals, alpha):
    """ When missing values arise, increases all counts by alpha (eg, missing values
        are now seen once. 1->2, 2->3 etc.

        Args:
            n_attr_obs: number of attribute observations
            n_class_obs: number of class observations
            n_attr_vals: number of attribute values
            alpha: laplace smoothing constant
            

        Returns:
            Smoothed probability for that attribute and class
    """
    return (n_attr_obs + alpha) / (n_class_obs + alpha * n_attr_vals)

def train_discrete_standard(df, class_attr, class_vals, d_attr, eps=0):
    """ For training Naive Bayes on a discrete attribute, with no / simple 
        smoothing.

        For each class value computes the conditional probability of observing 
        each attribute value given that class value. 

        Args:
            df: a pd.DataFrame
            class_attr: the column in df of the class to predict
            class_vals: the possible values of class_attr
            d_attr: the column in df of the discrete attribute
            eps: (optional) a value to replace any zero probabilities with.

        Returns:
            A dictionary, keyed by class values, of dictonaries storing the 
            conditional probability of observing each attribute value
    """

    attr_vals = df[d_attr].unique()

    params = dict()
    for cv in class_vals:

        params[cv] = dict()
        cv_obs = df[df[class_attr] == cv]
        for av in attr_vals:
            num_av_obs = np.count_nonzero((cv_obs[d_attr] == av).to_numpy())

            pval = num_av_obs / cv_obs.shape[0]

            params[cv][av] = pval if pval != 0 else eps

    return params

def train_discrete_laplace(df, class_attr, class_vals, d_attr, alpha=1):
    """ For training Naive Bayes on a discrete attribute, with laplace smoothing

        For each class value computes the conditional probability of observing 
        each attribute value given that class value. 

        Args:
            df: a pd.DataFrame
            class_attr: the column in df of the class to predict
            class_vals: the possible values of class_attr
            d_attr: the column in df of the discrete attribute
            alpha: the alpha value in laplace smoothing

        Returns:
            A dictionary, keyed by class values, of dictonaries storing the 
            conditional probability of observing each attribute value
    """

    attr_vals = df[d_attr].unique()

    params = dict()
    for cv in class_vals:

        params[cv] = dict()
        cv_obs = df[df[class_attr] == cv]
        for av in attr_vals:
            num_av_obs = np.count_nonzero((cv_obs[d_attr] == av).to_numpy())
            params[cv][av] = laplace_smoothing(num_av_obs, cv_obs.shape[0], len(attr_vals), alpha)

    return params

def train_gaussian(df, class_attr, class_values, num_attr):
    """ For training Naive Bayes on a numeric attribute.
    
        Calculates the mean and standard deviation of a numeric attribute for 
        each class. 

        Args:
            df: A pd.DataFrame 
            class_attr: the column in df of the class we are predicting
            class_values: an iterable of each possible value of class_attr
            num_attr: a numeric attribute in df

        Returns:
            A dictionary (keyed by the class values) of tuples (mean, std)
    """

    means = np.empty(len(class_values))
    stdevs = np.empty(len(class_values))
    for i, cv in enumerate(class_values):
        # A Series which is True wherever cv was observed and false otherwise
        cv_obs = df[class_attr] == cv

        # TODO: write our own functions?
        # by default Series.mean and Series.std will skip missing vals
        means[i] = df[num_attr][cv_obs].mean()
        stdevs[i] = df[num_attr][cv_obs].std()

    return means, stdevs

def train(df, discrete_attrs, numeric_attrs, class_name,
          train_discrete=train_discrete_standard, train_numeric=train_gaussian):
    """ Produce a Naive Bayes model for a given dataset.
    
        Args:
            df: a pd.DataFrame of training data
            discrete_attrs: a list of discrete attribute names of df
            numeric_attrs: a list of numeric attribute names of df
            class_name: the name in df of the class to predict
            train_discrete: a callable 
                train_discrete(df, class_name, class_vals, attr) which calculates
                the conditional probabilities of a discrete attribute attr
            train_numeric: a callable 
                train_numeric(df, class_name, class_vals, attr) which calculates
                the means and standard deviations of a numeric attribute attr
        
        Returns:
            A NBModel object
    """
    class_vals = df[class_name].unique()

    discrete = dict()
    for da in discrete_attrs:
        discrete[da] = train_discrete(df, class_name, class_vals, da)

    numeric = dict()
    for na in numeric_attrs:
        numeric[na] = train_numeric(df, class_name, class_vals, na)

    class_priors = discrete_probabilities(df[class_name].to_numpy(), class_vals)

    return NBModel(discrete, numeric, class_vals, class_priors)


In [ ]:
def guassian_pdf(x, mu, sigma):
    """Gaussian probability density function. Assumes a gaussian distribution for numeric attributes and calculates 
        the probabilities based on the density function
        Args:
            x: attribute value
            mu: Attribute mean
            sigma: Attribute Standard Deviation
        Returns:
            Likely hood of observing x
    """
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * (((x - mu) / sigma) ** 2))

def loglim(x):
    """ Returns log(x) for positive x and -float("inf") otherwise"""
    return log(x) if x > 0 else -float("inf")

def predict(df, nbm):
    """ Predict class labels using a Naive Bayes model.

        Args:
            df: a pd.DataFrame storing training instances.
            nbm: a NBModel object trained to predict on instances in df
        Returns:
            A pd.Series of class labels, with index equal to df.index
    """

    # numeric and discrete attributes
    n_attrs = list(nbm.numeric.keys())
    d_attrs = list(nbm.discrete.keys())

    predictions = pd.Series(
        np.empty(len(df), 
        dtype=nbm.class_vals.dtype), 
        index=df.index
    )

    for idx, row in df.iterrows():

        class_likelyhoods = np.empty(len(nbm.class_vals))
        for i, cv in enumerate(nbm.class_vals):

            cl = loglim(nbm.class_priors[i])

            if d_attrs:
                for a, x in zip(d_attrs, row[d_attrs]):

                    # TODO: this is not correct, need to change training?
                    if pd.notna(x) and x in nbm.discrete[a][cv]:
                        cl += loglim(nbm.discrete[a][cv][x])

            if n_attrs:
                for a, x in zip(n_attrs, row[n_attrs]):
                    if pd.notna(x):
                        # means and standard deviations for this numeric attribute
                        means, stdevs = nbm.numeric[a]
                        cl += loglim(guassian_pdf(x, means[i], stdevs[i]))

            class_likelyhoods[i] = cl

        predictions[idx] = nbm.class_vals[np.argmax(class_likelyhoods)]
    return predictions

In [ ]:
# This function should evaluate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(truth_labels, predictions, f_score_beta=1):
    """ Evaluates a prediction compared to the ground truth labels according to 
        a number of different metrics."""
    assert (len(truth_labels) == len(predictions))
    a = accuracy(truth_labels, predictions)
    cm = confusion_matrix(truth_labels, predictions)
    p = precision(cm)
    r = recall(cm)
    f = f_score(p, r, f_score_beta)
    return a, p, r, f

def accuracy(class_col, ybar):
    results = np.array(class_col) == ybar
    return np.count_nonzero(results == True) / len(ybar)

def precision(cm):
    """Precision of each class, returned as an average weighted by the number of
    instances in each class"""
    fp = np.sum(cm, axis=0)
    precisions = np.diag(cm) / np.where(fp == 0, 1, fp)
    weights = np.sum(cm, axis=1) / cm.sum()
    return np.sum(precisions * weights)

def recall(cm):
    """Recall of each class, returned as an average weighted by the number of
    instances in each class"""
    fp = np.sum(cm, axis=1)
    recalls = np.diag(cm) / np.where(fp == 0, 1, fp)
    weights = np.sum(cm, axis=1) / cm.sum()
    return np.sum(recalls * weights)

def f_score(p, r, beta):
    return ((1 + beta * beta) * p * r) / ((beta * beta * p) + r)

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer as KBD
def discretise_equal_width(numeric_series, nbins):
    """ Discretises a numeric data series accoding to equal-width discretisation
        
        Assumes no missing values.

        Args:
            numeric_series: a pd.Series containing numeric data
            nbins: the number of discrete classes for the data
            
        Returns:
            A pd.Series with nbins unique values.
    """

    est = KBD(n_bins=nbins, encode='ordinal', strategy='uniform')
    est.fit(numeric_series)
    return pd.DataFrame(est.transform(numeric_series), dtype='int64', columns=numeric_series.columns)


def discretise_k_means(numeric_series, k):
    """ Discretises a numeric data series according to k-means 

        Assumes no missing values.

        Args:
            numeric_series: a pd.Series containing numeric data
            k: the number of discrete categories
            repeates: the number of repeats of k-means
        
        Returns:
            A pd.Series with k unique values of the same index as numeric_series
    """

    est = KBD(n_bins=k, encode='ordinal', strategy='kmeans')
    est.fit(numeric_series)
    return pd.DataFrame(est.transform(numeric_series), dtype='int64', columns=numeric_series.columns)



### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

In [ ]:
def classify_zero_r(training_class_obs, num_to_classify):
    """ Classifies according to the most common class in the training data.

        Args:
            training_class_obs: the observations of class in the training data
            num_to_classify: the number of instances to classify

        Returns:
            A numpy array of length num_to_classify containing the most frequent
            value in training_class_obs
    """
    values, counts = np.unique(training_class_obs, return_counts=True)
    ind = np.argmax(counts)
    return np.full(num_to_classify, values[ind])

def classify_random(training_class_obs, num_to_classify):
    """ Classifies data at random according to the relative frequencies of 
        each class in the training observations.

        Args:
            training_class_obs: the observations of class in the training data
            num_to_classify: the number of instances to classify

        Returns:
            An array of length num_to_clasify of class values chosen at the 
            same probability as in training_class_obs
    """
    class_vals = np.unique(training_class_obs)
    class_probs = nb.discrete_probabilities(training_class_obs, class_vals)
    return np.random.choice(class_vals, size=num_to_classify, p=class_probs)

def classify_uniform(training_class_obs, num_to_classify):
    """ Classifies data uniformly at random.

        Args:
            training_class_obs: the observations of class in the training data
            num_to_classify: the number of instances to classify

        Returns:
            An array of length num_to_clasify of class values where each value
            was chosen with uniform probability.
    """
    class_vals = np.unique(training_class_obs)
    return np.random.choice(class_vals, size=num_to_classify)

def classify_one_r(training_df, class_name, testing_df):
    """ Classifies testing_df instances using a One-R classifier trained on 
        training_df

        Args:
            training_df: pd.DataFrame of training instances
            class_name: the name of the column of the class in training_df
            testing_df: a pd.DataFrame of test instances

        Returns:
            A pd.Series of class predictions with the same index as testing_df
    """

    #predict the most common class when an attribute is missing (Zero-R)
    values, counts = np.unique(training_df[class_name], return_counts=True)
    ind = np.argmax(counts)
    most_common_class = values[ind]

    #find the attribute which predicts class wih the lowest error rate
    min_error_rate = float("inf")
    best_attr = None
    best_predictor = None
    for attr in training_df.columns.drop(class_name):

        attr_groups = training_df[[attr, class_name]].groupby(attr).groups

        attr_predictor = {np.nan: most_common_class}
        for av, idx in attr_groups.items():
            
            #get most frequent class for av, choosing randomly if more than one
            most_frequent_class = training_df[class_name][idx].mode()
            choice = np.random.randint(len(most_frequent_class))

            attr_predictor[av] = most_frequent_class.iloc[choice]

        #check error rate of this attribute as predictor against training data
        attr_predictions = np.empty(len(training_df), dtype=training_df[class_name].dtype)
        for i, av in enumerate(training_df[attr]):
            attr_predictions[i] = attr_predictor[av]

        error_rate = np.count_nonzero(attr_predictions == training_df[class_name])

        if error_rate < min_error_rate:
            min_error_rate = error_rate
            best_attr = attr
            best_predictor = attr_predictor
    
    test_predictions = pd.Series(
        np.empty(len(testing_df), dtype=training_df[class_name].dtype), 
        index=testing_df.index)

    for i, row in testing_df.iterrows():
        test_predictions[i] = best_predictor[row[best_attr]]
    
    return test_predictions
    

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

In [ ]:
def k_split(total, k):
    """ basically calculated the indices by which to slice the dataset into k partitions"""
    quotient, remainder = divmod(total, k)
    indices = [quotient + 1] * remainder + [quotient] * (k - remainder)
    new = [indices[0]]
    for i in range(1, k - 1):
        new.append(new[i - 1] + indices[i])
    return new

def partition(df, k):
    """Splits the dataset into k partitions, and then allocates each in turn as the test set,
        whilst the remainder are used for training"""
    partition_lengths = k_split(df.shape[0], k)
    partitions = np.array_split(df, partition_lengths)
    splits = list()
    for i in range(k):
        train = pd.concat(partitions[:i] + partitions[i + 1:])
        test = partitions[i]
        splits.append((train, test))
    return splits


def cross_validation(df, cnfg, k):
    results = np.zeros((4, 1))
    for train, test in partition(df, k):
        model = nb.train(train, cnfg["discrete"], cnfg["numeric"], cnfg["class_col"])
        predictions = nb.predict(test, model)
        truth = test[cnfg["class_col"]]
        a, p, r, f = ev.evaluate(truth, predictions, print=False)
        results[0] += a
        results[1] += p
        results[2] += r
        results[3] += f
    return results / k

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

In [ ]:
def laplace_smoothing(n_attr_obs, n_class_obs, n_attr_vals, alpha):
    return (n_attr_obs + alpha) / (n_class_obs + alpha * n_attr_vals)

def train_discrete_laplace(df, class_attr, class_vals, d_attr, alpha=1):
    """ For training Naive Bayes on a discrete attribute, with laplace smoothing

        For each class value computes the conditional probability of observing 
        each attribute value given that class value. 

        Args:
            df: a pd.DataFrame
            class_attr: the column in df of the class to predict
            class_vals: the possible values of class_attr
            d_attr: the column in df of the discrete attribute
            alpha: the alpha value in laplace smoothing

        Returns:
            A dictionary, keyed by class values, of dictonaries storing the 
            conditional probability of observing each attribute value
    """

    attr_vals = df[d_attr].unique()

    params = dict()
    for cv in class_vals:

        params[cv] = dict()
        cv_obs = df[df[class_attr] == cv]
        for av in attr_vals:
            num_av_obs = np.count_nonzero((cv_obs[d_attr] == av).to_numpy())
            params[cv][av] = laplace_smoothing(num_av_obs, cv_obs.shape[0], len(attr_vals), alpha)

    return params

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.